# Data exploration

In [7]:
import pandas as pd


df = pd.read_csv("data/data.csv")
df.head()

,chain_id,first_res,input,dssp8
0,4trtA,1,VMKANVTKKTLNEGLGLLERVIPSRSSNPLLTALKVETSEGGLTLS...,CEEEEEEHHHHHHHHHHHHHHSCSCCSSTTTTEEEEEECSSEEEEE...
1,5g0mA,19,PVRVGLSVDASALGHTIPPDYTGLSYEQAQMANPNYFSGANTQLAG...,CEEEEEEEEEEEEEEEEPTTCCEEEEEGGGGGCTTTSSTTCHHHHH...
2,5l6mF,2,AYVLDTNVAIHLRDGDPEVTTRVTALNGAILLSIISRVELEGGVYR...,CEEECHHHHHHHHTTCHHHHHHHHHCCSCEEEEHHHHHHHHHHHTS...
3,1gytE,1,MEFSVKSGSPEKQRSACIVVGVFEPRRLSPIAEQLDKISDGYISAL...,CEEEEECCCGGGCCCSCEEEEEETTTEECHHHHHHHHHHTSHHHHH...
4,1h0hL,1,SKGFFVDTTRCTACRGCQVACKQWHGNPATPTENTGFHQNPPDFNF...,CEEEEEEGGGCCCCCHHHHHHHHHHTPPPCCCCCSSCSCCSSSCBT...


In [3]:
print("Number of data points: ", len(df))

Number of data points:  18731


In [30]:
secondary_structures = {"G": "310 helix", "H": "α-helix", "I": "π-helix", "E": "β-sheet", "B": "β-bridge", "T": "helix turn", "S": "bend", "C": "coil"}
amino_acid_key_to_name = {"A": "Alanine", "R": "Arginine", "N": "Asparagine", "D": "Aspartic acid", "C": "Cysteine", "Q": "Glutamine", "E": "Glutamic acid", "G": "Glycine", "H": "Histidine", "I": "Isoleucine", "L": "Leucine", "K": "Lysine", "M": "Methionine", "F": "Phenylalanine", "P": "Proline", "S": "Serine", "T": "Threonine", "W": "Tryptophan", "Y": "Tyrosine", "V": "Valine"}

df["Total secondary structures"] = df["dssp8"].apply(lambda x: len(x))

# Count the number of each secondary structure element in the dssp8 column for each row and store the counts in a new column.
for key in secondary_structures.keys():
    df[secondary_structures[key]] = df["dssp8"].apply(lambda x: x.count(key))

df.head()

,chain_id,first_res,input,dssp8,Total secondary structures,310 helix,α-helix,π-helix,β-sheet,β-bridge,helix turn,bend,coil,input_length
0,4trtA,1,VMKANVTKKTLNEGLGLLERVIPSRSSNPLLTALKVETSEGGLTLS...,CEEEEEEHHHHHHHHHHHHHHSCSCCSSTTTTEEEEEECSSEEEEE...,361,20,65,0,154,1,25,34,53,361
1,5g0mA,19,PVRVGLSVDASALGHTIPPDYTGLSYEQAQMANPNYFSGANTQLAG...,CEEEEEEEEEEEEEEEEPTTCCEEEEEGGGGGCTTTSSTTCHHHHH...,457,21,115,0,132,2,56,40,79,457
2,5l6mF,2,AYVLDTNVAIHLRDGDPEVTTRVTALNGAILLSIISRVELEGGVYR...,CEEECHHHHHHHHTTCHHHHHHHHHCCSCEEEEHHHHHHHHHHHTS...,127,4,71,0,16,1,11,7,17,127
3,1gytE,1,MEFSVKSGSPEKQRSACIVVGVFEPRRLSPIAEQLDKISDGYISAL...,CEEEEECCCGGGCCCSCEEEEEETTTEECHHHHHHHHHHTSHHHHH...,503,30,174,0,100,0,59,38,95,503
4,1h0hL,1,SKGFFVDTTRCTACRGCQVACKQWHGNPATPTENTGFHQNPPDFNF...,CEEEEEEGGGCCCCCHHHHHHHHHHTPPPCCCCCSSCSCCSSSCBT...,214,8,52,0,46,2,31,22,50,214


In [33]:
print("--- Secondary structure basic statistics ---")
# total secondary structures mean, min, max, std, median, 25th percentile, 75th percentile    
print("Total secondary structures mean: ", df["Total secondary structures"].mean())
print("Total secondary structures min: ", df["Total secondary structures"].min())
print("Total secondary structures max: ", df["Total secondary structures"].max())
print("Total secondary structures std: ", df["Total secondary structures"].std())
print("Total secondary structures median: ", df["Total secondary structures"].median())
print("Total secondary structures 25th percentile: ", df["Total secondary structures"].quantile(0.25))
print("Total secondary structures 75th percentile: ", df["Total secondary structures"].quantile(0.75))

print("---")
# average number of each secondary structure
for key in secondary_structures.keys():
    print(secondary_structures[key], "mean: ", df[secondary_structures[key]].mean())


print("--- Input sequence basic statistics ---")
# length of input sequence mean, min, max, std, median, 25th percentile, 75th percentile
df["input_length"] = df["input"].apply(lambda x: len(x))
print("Length mean: ", df["input_length"].mean())
print("Length min: ", df["input_length"].min())
print("Length max: ", df["input_length"].max())
print("Length std: ", df["input_length"].std())
print("Length median: ", df["input_length"].median())
print("Length 25th percentile: ", df["input_length"].quantile(0.25))
print("Length 75th percentile: ", df["input_length"].quantile(0.75))

# most common amino acids in input
amino_acids = "ACDEFGHIKLMNPQRSTVWY"
for aa in amino_acids:
    print("Amino acid ", aa, " count: ", df["input"].apply(lambda x: x.count(aa)).sum())


print("--- Basic correlations between input and secondary sequences ---")
# correlation between length of input and number of secondary structures
print("Correlation between input length and total secondary structures: ", df["input_length"].corr(df["Total secondary structures"], method="pearson"))

# Count of amino acids and occurrence of each secondary structure kind
correlations = []

for aa in amino_acids:
    for key in secondary_structures.keys():
        corr_secondary = df["input"].apply(lambda x: x.count(aa)).corr(df[secondary_structures[key]], method="pearson")
        correlations.append((f"Correlation between amino acid {amino_acid_key_to_name[aa]} count and {secondary_structures[key]} count", corr_secondary))

correlations.sort(key=lambda x: x[1], reverse=True)

for desc, corr in correlations[:]:
    print(f"{desc}: {corr}")

--- Secondary structure basic statistics ---
Total secondary structures mean:  213.95841119000588
Total secondary structures min:  39
Total secondary structures max:  1723
Total secondary structures std:  145.53981051358716
Total secondary structures median:  172.0
Total secondary structures 25th percentile:  108.0
Total secondary structures 75th percentile:  286.0
---
310 helix mean:  7.623405050451124
α-helix mean:  70.62393892477711
π-helix mean:  1.1827985692168064
β-sheet mean:  45.718167743313224
β-bridge mean:  2.385884362820992
helix turn mean:  23.933852971010623
bend mean:  19.61235385190326
coil mean:  39.17030590998879
--- Input sequence basic statistics ---
Length mean:  213.95841119000588
Length min:  39
Length max:  1723
Length std:  145.53981051358716
Length median:  172.0
Length 25th percentile:  108.0
Length 75th percentile:  286.0
Amino acid  A  count:  330590
Amino acid  C  count:  38960
Amino acid  D  count:  235280
Amino acid  E  count:  275057
Amino acid  F  coun